In [1]:
import numpy as np
import pandas as pd

# input data

In [36]:
age_start  = 50
screen = 0     # Clinically detected = 0, screen detected = 1 (webのsymptoms -> 0)
size = 50    # Tumour size mm
grade= 3     # Tumour grade
nodes = 5     # Number positive nodes
er = 1     # ER+ = 1, ER- = 0
her2 = 1     # HER2+ = 1, HER2- = 0, missing = 9
ki67 = 1     # KI67+ = 1, KI67- = 0, missing = 9
generation = 3     # Chemo generation 0, 2 or 3 only
horm = 1     # Hormone therapy Yes = 1, no = 0
traz = 1     # Trastuzumab therapy Yes = 1, no = 0
bis = 1     # Bisphosphonate therapy Yes = 1, no = 0
radio = 0     # Radiotherapy Yes = 1, no = 0
r_enabled  =  0     # Radiotherapy enabled = 1, disabled = 0

In [37]:
#screen gradeの調整
screen =  0.204 if screen == 2 else screen
grade = 2.13 if grade == 9 else grade
max_time = 15
time = np.array(range(1,max_time+1))

screen,grade


(0, 3)

# 層別化

$$ 
\begin{align}
    ER+ : \ H_c(t)  &= exp\big(0.7424402 - \frac{7.527762}{\sqrt{t}} - 1.812513 \times \frac{log(t)}{\sqrt{t}}\big) \\\\
    ER- : \ H_c(t)  &= exp\big(-1.156036 + \frac{0.4707332}{t^2} - \frac{3.51355}{t}\big)
\end{align}
$$

In [38]:
her2_beta = 0.2413 if her2 == 1 else -0.0762 if her2 == 0 else 0


if er == 1:
    grade_val = grade
    age_mfp_1 = (age_start/10)**-2-0.0287449295
    age_beta_1 = 34.53642
    age_mfp_2 = (age_start/10)**-2*np.log(age_start/10)-0.0510121013
    age_beta_2 = -34.20342
    size_mfp = np.log(size/100)+1.545233938
    size_beta = 0.7530729
    nodes_mfp = np.log((nodes+1)/10)+1.387566896
    nodes_beta = 0.7060723
    grade_beta = 0.746655
    screen_beta = -0.22763366
    ki67_beta = 0.14904 if ki67 == 1 else -0.11333 if ki67 == 0 else 0
    
else:
    grade_val =  1 if grade == 2 or grade ==3 else 0
    age_mfp_1 = age_start-56.3254902
    age_beta_1 = 0.0089827
    age_mfp_2 = 0
    age_beta_2 = 0
    size_mfp = (size/100)**0.5-0.5090456276
    size_beta = 2.093446
    nodes_mfp = np.log((nodes+1)/10)+1.086916249
    nodes_beta = 0.6260541
    grade_beta = 1.129091
    screen_beta = 0
    ki67_beta = 0
    
    

In [39]:
r_prop = 0.69
r_breast = 0.82
r_other = 1.07
if r_enabled == 1:
    r_base_br = np.log(1/((1-r_prop) + r_prop*r_breast))
    r_base_oth = np.log(1/((1-r_prop) + r_prop*r_other))
else:
    r_base_br = 0
    r_base_oth = 0

In [40]:
# Other mortality prognostic index (mi)
mi = 0.0698252*((age_start/10)**2-34.23391957) + r_base_oth

# Breast cancer mortality prognostic index (pi)
pi = age_beta_1*age_mfp_1 + age_beta_2*age_mfp_2 + size_beta*size_mfp + \
  nodes_beta*nodes_mfp + grade_beta*grade_val + screen_beta*screen + \
  her2_beta + ki67_beta + r_base_br

In [50]:
c = 0 if generation == 0 else -0.248 if generation == 2 else -0.446
h = -0.3857 if horm and er == 1 else 0
h10 = [h for i in range(10)] + [h-0.26 for i in range(len(time)-10)]
t = -0.3567 if her2 == 1 and traz == 1 else 0
b = -0.198 if bis == 1 else 0

In [52]:
if r_enabled == 1:
    r_br = np.log(r_breast) if radio == 1 else 0
    r_oth = np.log(r_other) if radio == 1 else 0
else:
    r_br = 0
    r_oth = 0

In [60]:
h,c,b,r_br,t

(-0.3857, -0.446, -0.198, 0, -0.3567)

In [ ]:
class Patient(input_data,event):
    def __init__(self,input_data, event):
        self.age =  input_data["age"]
        if generation == 2:
            c = -0.248
        else:
            c = generation * -0.446
        
    def predict(self, h10=False):
        self.__print_status()
        
        if self.er == 0:
            
            self.__predict_wo_er()
        else:
            self.__predict_w_er()
            pass
    
    def __predict_wo_er(self):
    
    def __predict_w_er(self):
        
    def __print_status(self):